In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import os
import cv2

In [38]:
train = pd.read_csv('./tensorflow-great-barrier-reef/train-5folds.csv')
train['pos'] = train.annotations != '[]'
fold = -2

In [39]:
val = train[train['fold'] == 1]


In [40]:
val["num_bbox"] = val['annotations'].apply(lambda x: str.count(x, 'x'))

C:\Users\rajag\AppData\Local\Temp/ipykernel_14144/3150761237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val["num_bbox"] = val['annotations'].apply(lambda x: str.count(x, 'x'))


In [41]:
val[val['num_bbox'] > 3].image_path.to_list()[0]

'../input/tensorflow-great-barrier-reef/train_images/video_0/4558.jpg'

In [42]:
image_paths = val.image_path.tolist()

In [43]:
image_paths[2135]

'../input/tensorflow-great-barrier-reef/train_images/video_0/8970.jpg'

In [44]:
HOME_DIR = "./yolo_data_subseq_empty_all"
DATASET_PATH = os.path.join(f'fold{str(fold)}', 'images')
LABELS_PATH = os.path.join(f'fold{str(fold)}', 'labels')

os.makedirs(os.path.join(HOME_DIR, f'fold{str(fold)}'))

for path in [DATASET_PATH, LABELS_PATH]:
    os.makedirs(os.path.join(HOME_DIR, path))
    os.makedirs(os.path.join(HOME_DIR, path, 'train'))
    os.makedirs(os.path.join(HOME_DIR, path, 'val'))

In [45]:
sharp_filter = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
def gamma_enhance(img, gamma=1.0):
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    return cv2.LUT(img, lookUpTable)

In [46]:
# # # Preprocessed Images

# for i in tqdm(range(len(train))):
#     row = train.loc[i]
#     img = cv2.imread(row.image_path)
#     img = gamma_enhance(cv2.filter2D(img, -1, sharp_filter), gamma=2)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     if row.video_id != fold:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg')
#     else:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg') 

In [47]:
annos = []
for i, x in train.iterrows():
    if x.fold == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        # if not x.pos: continue
        # val
    path = f'./{x.image_path[9:]}'
    img = cv2.imread(path)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = gamma_enhance(cv2.filter2D(img, -1, sharp_filter), gamma=2)
    cv2.imwrite(f'{HOME_DIR}/fold{fold}/images/{mode}/{x.image_id}.jpg', img)
    # copyfile(f'./tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
    #             f'{HOME_DIR}/fold{fold}/images/{mode}/{x.image_id}.jpg')
    r = ''
    anno = eval(x.annotations)
    if not x.pos:
        continue
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'{HOME_DIR}/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [49]:
annos = []
fold = 2
val = train[train['fold'] == fold]
for i, x in val.iterrows():
    mode = 'val'
    path = f'./{x.image_path[9:]}'
    img = cv2.imread(path)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = gamma_enhance(cv2.filter2D(img, -1, sharp_filter), gamma=2)
    cv2.imwrite(f'{HOME_DIR}/fold-2/images/{mode}/{x.image_id}.jpg', img)
    # copyfile(f'./tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
    #             f'{HOME_DIR}/fold-1/images/{mode}/{x.image_id}.jpg')
    r = ''
    anno = eval(x.annotations)
    if not x.pos:
        continue
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'{HOME_DIR}/fold-2/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)